In [5]:
pip install opencv-python mediapipe numpy playsound


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7044 sha256=0cc594273474bab4b2e5df8c6d6fccd47ed6576518d4bc678b2c3514a61152ee
  Stored in directory: c:\users\deepthi\appdata\local\pip\cache\wheels\cf\42\ff\7c587bae55eec67b909ca316b250d9b4daedbf272a3cbeb907
Successfully built playsound
Note: you may need to restart the kernel to use updated packages.


In [18]:
import cv2
import mediapipe as mp
import numpy as np
from playsound import playsound

# Initialize FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

# Landmark indices
LEFT_EYE = [362, 385, 387, 263, 373, 380]
RIGHT_EYE = [33, 160, 158, 133, 153, 144]
UPPER_LIP = 13
LOWER_LIP = 14

# Eye aspect ratio (EAR) function
def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    return (A + B) / (2.0 * C)

# Thresholds
EAR_THRESH = 0.25
MOUTH_THRESH = 25
frame_check = 20
eye_flag = 0
mouth_flag = 0

vid = cv2.VideoCapture(0)

while True:
    ret, frame = vid.read()
    if not ret:
        break

    h, w = frame.shape[:2]
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    if results.multi_face_landmarks:
        for landmarks in results.multi_face_landmarks:
            mesh = np.array([(int(p.x * w), int(p.y * h)) for p in landmarks.landmark])

            left_eye = mesh[LEFT_EYE]
            right_eye = mesh[RIGHT_EYE]
            left_ear = eye_aspect_ratio(left_eye)
            right_ear = eye_aspect_ratio(right_eye)
            ear = (left_ear + right_ear) / 2.0

            mouth_open = np.linalg.norm(mesh[LOWER_LIP] - mesh[UPPER_LIP])

            # Draw features
            cv2.polylines(frame, [left_eye], True, (0, 255, 0), 1)
            cv2.polylines(frame, [right_eye], True, (0, 255, 0), 1)
            cv2.circle(frame, tuple(mesh[UPPER_LIP]), 2, (255, 0, 0), -1)
            cv2.circle(frame, tuple(mesh[LOWER_LIP]), 2, (255, 0, 0), -1)

            # Drowsiness
            if ear < EAR_THRESH:
                eye_flag += 1
                if eye_flag >= frame_check:
                    cv2.putText(frame, "DROWSINESS DETECTED", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    playsound(r"C:\Users\deepthi\Downloads\ElevenLabs_Text_to_Speech_audio.mp3",) 
            else:
                eye_flag = 0

            # Yawning
            if mouth_open > MOUTH_THRESH:
                mouth_flag += 1
                if mouth_flag >= frame_check // 2:
                    cv2.putText(frame, "YAWNING DETECTED", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
                    playsound(r"C:\Users\deepthi\Downloads\ElevenLabs_Text_to_Speech_audio.mp3",) 
            else:
                mouth_flag = 0

    cv2.imshow("Drowsiness & Yawning Detector", frame)
    if cv2.waitKey(1) & 0xFF == ord("n"):
        break

vid.release()
cv2.destroyAllWindows()
